# Finished notebook
#### What do we learn from it?
## 1. Functions:

1) test_split_size_effect:

        Splits the dataset, creates training graph, runs the LP methods on the test set

        Creates ./CSV/auc_prediction_rav.csv in the process

2) test_on_non_splits:

        First, runs MIDAS on the entire dataset. Then, splits the dataset and updates the test sores with LP

        Creates ./CSV/rav_auc_split.csv

3) create_auc_clique_n:

        Splits the data, constructs training graphs, imputes a clique into the test set only

        Runs MIDAS and MIDAS+LP on the test set

        Creates ./CSV/rav_auc_clique_'+str(n)+'.csv, where n is the number of nodes taking part in the clique


## 2. CSVs created:

- ./CSV/auc_prediction_rav.csv

- ./CSV/rav_auc_split.csv

- ./CSV/rav_dataset_info.csv

        For each split of the dataset, lists the exact train and test sizes, as well as the anomaly train and test sizes

- rav_auc_clique_50.csv, where 50 is the number of imputed anomaly edges (clique)


## 3. Some code to prove the superiority/correctness of our approaches:

- Splitting works

- sum(y) is faster than y.count(1)

- .intersection() or .union() are faster than nx.jaccard_coefficient

## 4. Some "edge ranking" stuff that seems highly unnecessary

In [2]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.12.0
ipykernel        : 6.19.2
ipywidgets       : 8.1.2
jupyter_client   : 8.1.0
jupyter_core     : 5.3.0
jupyter_server   : 1.23.4
jupyterlab       : not installed
nbclient         : 0.5.13
nbconvert        : 6.5.4
nbformat         : 5.7.0
notebook         : 6.5.4
qtconsole        : not installed
traitlets        : 5.7.1


In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from itertools import combinations
from sklearn.metrics import roc_auc_score
import time
from tqdm import tqdm, trange
from Midas import FilteringCore, NormalCore, RelationalCore
from random import uniform, randint

from thesis_library import *

In [ ]:
# def evaluate_methods(dataset: str, test_size: float, threshold=10) -> tuple:
#     '''
#     Returns a triple of the AUC scores of the following methods:
#     Common Neighbours, Jaccard Coefficient, Preferential Attachment
    
#     If the case imbalance in the test set is too extreme (threshold param), returns None, None, None instead
#     '''
    
#     print("Reading data")
#     data, label = read_data(dataset = dataset, plant='clique')
    
#     print("Splitting data")
#     X_train, X_test, y_train, y_test = split(data, label, test_size=test_size)
    
#     print("Constructing the training graph (anomalies disallowed)")
#     G = construct_training_graph(X_train, y_train, True, False)
    
#     print("Filtering the test set")
#     X_test, y_test = filter_test(X_test, y_test, G)
    
#     #AUC / ROC is ill-defined with an extreme case imbalance
#     if sum(y_test) > threshold: #sum(y_test) is the number of anomalous edges
        
#         print("Invoking the Link Prediction methods \n")
#         print("Dataset length:   ", len(y_test))
#         print("Anomalies present:", sum(y_test))
        
#         cn = apply_lp('Common Neighbours', [1]*len(y_test), X_test, G)
#         jc = apply_lp('Jaccard Coefficient', [1]*len(y_test), X_test, G)
#         pa = apply_lp('Preferential Attachment', [1]*len(y_test), X_test, G)
        
#         cn, jc, pa = roc_auc_score(y_test, cn), roc_auc_score(y_test, jc), roc_auc_score(y_test, pa)
        
#         cn, jc, pa = round(cn, 4), round(jc, 4), round(pa, 4)
        
#         return cn, jc, pa, len(y_test), sum(y_test)
        
#     else:
#         print("Not enough anomalous edges in the test set, only", sum(y_test))
#         return -1, -1, -1, 0, 0

In [ ]:
#cn_auc, jc_auc, pa_auc, _, _ = evaluate_methods(dataset="ISCX", test_size=0.1)

# print('')
# print("Common Neighbours AUC:      ", cn_auc)
# print("Jaccard Coefficient AUC:    ", jc_auc)
# print("Preferential Attachment AUC:", pa_auc)

In [ ]:
# def test_split_size_effect(df = None) -> None:
#     '''
#     Runs the evaluate_methods code for all 1:9 up to 9:1 train-test-split settings
#     Saves the results to a dataframe called "auc_prediction_rav.csv"
#     If the dataframe has already been created, pass it as an argument to append to it
#     Otherwise, the results may be overwritten!
#     '''
    
#     if df is None:
#         df = pd.DataFrame({"Split (train:test)": [], "Dataset": [], "Test size (overall)": [], \
#                            "Test size (anomaly)": [], "Method": [], "AUC": []})
        
#     for dataset in DATASETS:
#         for test_size in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
            
#             print(dataset, test_size)
            
#             #Just for display purposes:
#             split_name = get_split_name(test_size)
            
#             #Getting the AUCs:
#             cn_auc, jc_auc, pa_auc, n, n_anomaly = evaluate_methods(dataset=dataset, test_size=test_size)
            
#             #Saving the results into the resul dataframe:
#             df.loc[df.shape[0]] = [split_name, dataset, n, n_anomaly, "Common neighbours", cn_auc]
#             df.loc[df.shape[0]] = [split_name, dataset, n, n_anomaly, "Jaccard coefficient", jc_auc]
#             df.loc[df.shape[0]] = [split_name, dataset, n, n_anomaly, "Preferential attachment", pa_auc]
            
#             #Saving the intermediate results and cooling down the processor:
#             df.to_csv("./CSV/auc_prediction_rav.csv", index=False)
#             time.sleep(15)

### The pipeline of the above for a different test_size split (runs 3 hours):

In [ ]:
# df = pd.read_csv("./CSV/auc_prediction_rav.csv")
# test_split_size_effect(df=df)

### Obtaining the score data for pure Preferential Attachment for comparison

### Create dataset info has simply this single line to invoke:

In [ ]:
#create_dataset_info(DATASETS)

In [ ]:
# def impute_clique(X_test, y_test, n: int) -> tuple:
#     '''Perhaps it would be wiser to allow new nodes here?
#     Imputing a clique of 0 nodes equals to not doing anything at all! Use it!'''
    
#     final_t = X_test[-1][-1]+1
#     for i in range(n): #is starting from 0 okay?
#         for j in range(n):
#             if i != j:
#                 X_test.append([i, j, final_t])
#                 y_test.append(1)
            
#     return X_test, y_test

In [ ]:
# def create_auc_clique_n(n: int, midas_list: list, datasets: list, lp_methods: list, df_clique=None):
#     '''
#     1) Splits the data
#     2) Constructs training graph on the train split
#     3) Imputes an n-clique into the test data (pass n=0 for no imputation)
#     4) Runs MIDAS score on the test set
#     5) Updates it with Link Prediction scores
#     '''
    
#     assert midas_version in ["normal", 'relational'], "Use 'normal' or 'relational'"
    
#     if df_clique is None:
#         df_clique = pd.DataFrame(columns=["Split (train:test)", "Dataset", "Midas_version", "Method", "AUC", 'Time'])
    
#     print("Trying a total of", 9*len(midas_list)*len(datasets)*(len(lp_methods)+1), " combinations.")
    
#     ####### MAIN LOOP ###########
#     for dataset in datasets:
#         print("Reading dataset:", dataset)
#         X, y = read_data(dataset)
        
#         #Handling the proper MIDAS version and its display name:
#         for midas_name in midas_list:
#             if midas_name == 'normal':
#                 midas, midas_name = NormalCore(2, 1024), 'MIDAS'
#             elif midas_name == "relational":
#                 midas, midas_name = RelationalCore(2, 1024), 'MIDAS-R'
#             else:
#                 raise ValueError("MIDAS version not supported. Pass 'normal' or 'relational'.")
            
#             #Handling the proper train-test split size:
#             for test_size in [round(0.1*(i+1), 2) for i in range(0, 9)]:

#                 #Splitting the data:
#                 X_train, X_test, y_train, y_test = split(X, y, test_size)

#                 G = construct_training_graph(X_train, y_train)

#                 X_test, y_test = impute_clique(X_test, y_test, n) #not too fast of an implementation

#                 #Setting up a proper midas version and its display name for the CSV filename:
#                 if midas_version == 'normal':
#                     midas, midas_version = NormalCore(2, 1024), "MIDAS"
#                 elif midas_version == 'relational':
#                     midas, midas_version = RelationalCore(2, 1024), "MIDAS-R"

#                 #Obtaining the results for pure MIDAS:
#                 score = [0.0] * len(y_test)
#                 t1 = time.time()

#                 for i in trange(len(y_test), desc=midas.nameAlg, unit_scale=True):
#                     score[i] = midas.Call(*X_test[i])
#                     #score[i] = midas.process_edge(*X_test[i])

#                 t2 = time.time()
#                 auc = roc_auc_score(y_test, score)

#                 df_clique.loc[df_clique.shape[0]] = [get_split_name(test_size), dataset, midas_version, \
#                                                      'none', auc, round(t2-t1, 4)]

#                 #Reinforcing the scores with LP methods:
#                 for method in lp_methods:
#                     t1 = time.time()
#                     method_score = apply_lp(method, score, X_test, G)
#                     auc = roc_auc_score(y_test, method_score)
#                     t2 = time.time()
#                     df_clique.loc[df_clique.shape[0]] = [get_split_name(test_size), dataset, midas_version, \
#                                                          method, auc, round(t2-t1, 4)]

#                 #Saving all the results:
#                 df_clique.to_csv('./CSV/rav_auc_clique_'+str(n)+'.csv', index=False)
#                 time.sleep(15)

#             time.sleep(40)

In [ ]:
# df = pd.read_csv('./CSV/rav_auc_clique_30.csv')
# create_acc_clique_n(0, ['normal', 'relational'], DATASETS, LP_METHODS, df=df)

In [ ]:
# def test_on_non_splits(midas_list: list, datasets: list, lp_methods: list, df=None) -> None:
#     '''First, runs a given MIDAS on the ENTIRE dataset
#     Then, splits the scores alongside the data to update them with LP scores
#     If no dataframe is passed, a new one is created and saved at ./CSV/rav_auc_split.csv'''
    
#     print("Trying a total of", 9*len(midas_list)*len(datasets)*(len(lp_methods)+1), "combinations.")
    
#     if df is None:
#         df = pd.DataFrame(columns=['Split (train:test)', 'Dataset', '#nodes', '#edges', 'MIDAS', 
#                                    'Method', 'AUC', 'Time [s]'])
    
#     #Handling the dataset loop:
#     for dataset in datasets:
#         print("Reading dataset", dataset)
#         X, y = read_data(dataset)
        
#         #Handling the MIDAS loop:
#         for midas_name in midas_list:
#             if midas_name == 'normal':
#                 midas, midas_name = NormalCore(2, 1024), 'MIDAS'
#             elif midas_name == "relational":
#                 midas, midas_name = RelationalCore(2, 1024), 'MIDAS-R'
#             elif midas_name == 'custom':
#                 print("Setting up the custom MIDAS.")
#                 midas, midas_name = MIDAS(4, 5, 3, is_switchboard=False), "Custom"
#             else:
#                 raise ValueError("MIDAS version not supported. Pass 'normal', 'relational', 'custom'.")
            
#             #Running MIDAS on the entire dataset - the scores will be split
#             score = [0.0] * len(y)
#             for i in trange(len(y), desc=midas.nameAlg, unit_scale=True):
#                 score[i] = midas.Call(*X[i])
#                 #score[i] = midas.process_edge(*X[i])
#             auc = roc_auc_score(y, score)
            
#             #Saving the MIDAS score anyway:
#             df.loc[df.shape[0]] = ["10:00", dataset, -1, -1, midas_name, 'none', auc, -1]
#             df.to_csv('./CSV/rav_auc_split.csv', index=False)
            
#             for test_size in [round(0.1*(i+1), 2) for i in range(0, 9)]:
                
#                 print("Processing", get_split_name(test_size))
                
#                 #Preparing the split:
#                 X_train, X_test, y_train, y_test, score_test = split(X, y, test_size, score)
#                 G = construct_training_graph(X_train, y_train, True, False) #Not saving anomalies in
                
#                 #Looping over the 3 available LP methods:
#                 for method in lp_methods: 
                    
#                     start_time = time.time()
#                     method_score = apply_lp(method, score_test, X_test, G)
#                     auc = roc_auc_score(y_test, method_score) if sum(y_test) !=0 else -1
#                     end_time = time.time()
                    
#                     df.loc[df.shape[0]] = [get_split_name(test_size), dataset, G.number_of_nodes(), G.number_of_edges(), 
#                                            midas_name, method, auc, round(end_time-start_time, 4)]
                    
#                     time.sleep(20)
                    
#                 df.to_csv('./CSV/rav_auc_split.csv', index=False)
                
#                 time.sleep(40)

In [ ]:
# df = pd.read_csv('./CSV/rav_auc_split.csv')
# test_on_non_splits(midas_list = ['normal', 'relational'], datasets = ['NYC_Taxi'], lp_methods=LP_METHODS, df=df)

In [12]:
def grand_test(midas_list: list, datasets: list, lp_methods: list, df=None) -> None:
    '''First, runs a given MIDAS on the ENTIRE dataset
    Then, splits the scores alongside the data to update them with LP scores
    If no dataframe is passed, a new one is created and saved at ./CSV/rav_test_on_splits.csv'''
    
    print("Trying a total of", 9*len(midas_list)*len(datasets)*(len(lp_methods)+1), "combinations.")
    
    if df is None:
        df = pd.DataFrame(columns=['Split (train:test)', 'Dataset', '#nodes', '#edges', 'MIDAS', 
                                   'Method', 'AUC', 'Runtime'])
    
    #Handling the dataset loop:
    for dataset in datasets:
        print("Reading dataset", dataset)
        X, y = read_data(dataset, plant='clique')
        
        #Handling the MIDAS loop:
        for midas_name in midas_list:
            if midas_name in ['normal', 'Normal', 'MIDAS']:
                midas, midas_name = NormalCore(2, 1024), 'MIDAS'
            elif midas_name in ['relational', 'Relational', 'MIDAS-R']:
                midas, midas_name = RelationalCore(2, 1024), 'MIDAS-R'
            elif midas_name in [None, 'None', 'none']:
                midas, midas_name = None, "None"
            else:
                raise ValueError("MIDAS version not supported. Pass 'normal', 'relational', 'none'.")
            
            #Running MIDAS on the entire dataset - the scores will be split
            if midas_name not in [None, 'None', 'none']:
                score = [0.0] * len(y)
                t1 = time.time()
                for i in trange(len(y), desc=midas.nameAlg, unit_scale=True):
                    score[i] = midas.Call(*X[i])
                t2 = time.time()
                time_taken = round(t2-t1, 4)
                auc = roc_auc_score(y, score)
            else:
                score, auc, time_taken = [1.0] * len(y), -1, 0
            
            #Saving the MIDAS score anyway:
            df.loc[df.shape[0]] = ["10:00", dataset, -1, -1, midas_name, 'None', auc, time_taken]
            df.to_csv('./CSV/rav_test_on_splits.csv', index=False)
            
            for test_size in [round(0.1*(i+1), 2) for i in range(0, 9)]:
                
                print("Processing", get_split_name(test_size))
                
                #Preparing the split:
                X_train, X_test, y_train, y_test, score_test = split(X, y, test_size, score)
                G = construct_training_graph(X_train, y_train, True, False) #Not saving anomalies in
                
                #Looping over the 3 available LP methods + the None LP method:
                for method in lp_methods+['None']: 
                    
                    t1 = time.time()
                    method_score = apply_lp(method, score_test, X_test, G)
                    auc = roc_auc_score(y_test, method_score) if sum(y_test) !=0 else -1
                    t2 = time.time()
                    
                    df.loc[df.shape[0]] = [get_split_name(test_size), dataset, G.number_of_nodes(), G.number_of_edges(), 
                                           midas_name, method, auc, round(t2-t1, 4)]
                    
                    time.sleep(20)
                    
                df.to_csv('./CSV/rav_test_on_splits.csv', index=False)
                
                time.sleep(40)

In [21]:
DATASETS[4:]

['Gowalla', 'NYC_Taxi']

In [18]:
lp_methods = ['Common Neighbours', 'Preferential Attachment']
df = pd.read_csv('./CSV/rav_test_on_splits.csv')
grand_test(['None'], DATASETS[4:], lp_methods, df=df)

Trying a total of 135 combinations.
Reading dataset DARPA
Processing 9:1


Common Neighbours: 100%|███████████████████████████████████████████████████████████| 455k/455k [00:12<00:00, 37.4kit/s]
Preferential Attachment: 100%|██████████████████████████████████████████████████████| 455k/455k [00:00<00:00, 456kit/s]


Processing 8:2


Common Neighbours: 100%|███████████████████████████████████████████████████████████| 909k/909k [00:26<00:00, 34.0kit/s]
Preferential Attachment: 100%|██████████████████████████████████████████████████████| 909k/909k [00:01<00:00, 468kit/s]


Processing 7:3


...

Processing 1:9


Common Neighbours: 100%|██████████████████████████████████████████████████████████| 2.29M/2.29M [00:06<00:00, 329kit/s]
Preferential Attachment: 100%|████████████████████████████████████████████████████| 2.29M/2.29M [00:04<00:00, 520kit/s]


Reading dataset CTU13
Processing 9:1


Common Neighbours:  73%|████████████████████████████████████████████▋                | 185k/252k [11:32<04:12, 267it/s]


KeyboardInterrupt: 

## Runtime proofs:

In [ ]:
#Confirmation the new splitting works

# X, y = read_data('ISCX')
# for test_size in [round(0.1*(i+1), 2) for i in range(0, 9)]:
#     X_train, X_test, y_train, y_test = split(X, y, test_size)
#     print(X_train[-1])
#     print(X_test[0])
#     print(len(X_test)/(len(X_train)+len(X_test)))
#     print('')

In [ ]:
time_nx_cn, time_pythonic_cn = [], []
time_nx_jc, time_pythonic_jc = [], []
time_nx_pa, time_pythonic_pa = [], []

for nr_iters in [10000, 50000, 100000, 500000, 1000000]:
    
    print("trying currently nr_iters:", nr_iters)
     G = nx.erdos_renyi_graph(250, 0.03)
    
    # COMMON NEIGHBOURS:
    #Pythonic:
    t1 = time.time()
    for _ in range(nr_iters):
        wow = len(set(G[1]).intersection(set(G[25])))
    t2 = time.time()
    time_pythonic_cn.append(t2 - t1)
    
    #Networkx:
    t1 = time.time()
    for _ in range(nr_iters):
        wow = len(tuple(nx.common_neighbors(G, 1, 25)))
    t2 = time.time()
    time_nx_cn.append(t2 - t1)
    
    # JACCARD COEFFICIENT:
    #Pythonic:
    t1 = time.time()
    for _ in range(nr_iters):
        wow = len(set(G[1]).intersection(set(G[25]))) / len(set(G[1]).union(set(G[25])))
    t2 = time.time()
    time_pythonic_jc.append(t2 - t1)
    
    #Networkx:
    t1 = time.time()
    for _ in range(nr_iters):
        wow = next(nx.jaccard_coefficient(G, [(1, 25)]))[2]
    t2 = time.time()
    time_nx_jc.append(t2 - t1)
    
    #PREFERENTIAL ATTACHMENT:
    # Pythonic:
    t1 = time.time()
    for _ in range(nr_iters):
        wow = len(G[1]) * len(G[25])
    t2 = time.time()
    time_pythonic_pa.append(t2 - t1)
    
    # Networkx:
    t1 = time.time()
    for _ in range(nr_iters):
        wow = tuple(nx.preferential_attachment(G, [(1, 25)]))[0][2]
    t2 = time.time()
    time_nx_pa.append(t2 - t1)
    
#### RESULTS ####
df_time = pd.DataFrame({'size': [10000, 50000, 100000, 500000, 1000000], 
                        'time_nx': time_nx, 'time_pythonic': time_pythonic})
df_time['method'] = ['Jaccard coefficient']*5 + ['Common neighbors']*5 + ['Preferential attachment']*5

## Some fuckery about edge ranking:

In [ ]:
# predict new edges

# 1. split dataset
# 2. make a graph for training dataset
#    - ignore the anomaly edges
# 3. process test dataset
#    - delete anomaly edges?
#    - delete edges already in train dataset
#    - delete edges wihich nodes are not in train dataset
# 3. for every two nodes in training dataset, compute cn,jc,pa
#    - ignore edges already in train dataset
#    - output a ranked list
# 4. take the first n pairs from ranked list, determine the size of the intersection

In [ ]:
def possible_edges_ranking(dataset: str):
    '''This code is more-or-less useless due to insane running time
    Returns a sorted dataframe of all possible edges and their scores
    This might be a massive dataset, so proceed with caution
    Does not filter out testing/training edges from the final dataframe
    
    Converting an iterator into a tuple works (For me) at almost 1M elements per second'''
    
    data, label = read_data(dataset=dataset)
    
    train, test, train_label, test_label = split(data, label, test_size = 0.5)

    G = construct_training_graph(train, train_label)

#     X_test, y_test = filter_test(test, test_label, G)

#     #Filtering out the anomalies from the test dataset
#     good_rows = [(edge[0], edge[1], label) for (edge, label) in zip(X_test, y_test) if label==0]
#     X_test, y_test = [(edge[0], edge[1]) for edge in good_rows], [edge[2] for edge in good_rows]

    df = pd.DataFrame(columns=['u', 'v', 'CommonNeighbours','JaccardCoefficient','PreferentialAttachment'])
    print("Total number of possible edges:", len(list(combinations(G.nodes,2))))
    
    #Getting the iterator objects of Jaccard and PrefAtt scores for all pairs of nodes in the G object
    jacc = nx.jaccard_coefficient(G)
    pref = nx.preferential_attachment(G)
    
    #Converting these massive generators into tuples might take massive amount of time:
    df['JaccardCoefficient'] = tuple(jacc)
    df['PreferentialAttachment'] = tuple(pref)
    
    #And calling the CommonNeighbours on every single pair of nodes might also be super time-consuming:
    df['CommonNeighbours'] = df['JaccardCoefficient'].apply(lambda x: len(tuple(nx.common_neighbors(G, x[0], x[1]))))
    
    #Lastly, retrieve the integer labels of the nodes (might as well be a large computation)
    df['u'] = df['JaccardCoefficient'].apply(lambda x: x[0])
    df['v'] = df['JaccardCoefficient'].apply(lambda x: x[1])
    
    df.sort_values(by = ['CommonNeighbours', 'JaccardCoefficient', 'PreferentialAttachment'], inplace=True)

    return df

In [ ]:
def single_dataset_plot(dataset: str, show_train=True, show_test=True):
    '''TO DO: decide whether this is even needed. 
    IMO it is not, for AUC_on_splits (plot_auc_and_split_per_dataset) does that too, but better'''
    
    #Using only the proper part of the dataset
    df, dataset_info = split_data_reader(dataset)
    
    reference_values = df[df['Method'] == 'No LP'].reset_index(drop=True)
    reference_values = reference_values['AUC'][0], reference_values['AUC'][1]
    
    plt.figure(figsize=(8,8), dpi=100)
    
    #The AUC plot:
    ax1 = plt.subplot(311)
    sns.pointplot(x = 'Split (train:test)', y = 'AUC', hue = 'Method', data = df, ax=ax1)
    
    ax1.axhline(y = reference_values[0], linestyle = '--') #MIDAS
    ax1.axhline(y = reference_values[1], linestyle = '--') #MIDAS-R
    
    plt.title('ROC/AUC in dataset ' + dataset)
    plt.xticks([])
    plt.xlabel('')
    plt.ylabel('AUC/ROC')
    
    #The anomaly % plot for training data:
    if show_train:
        ax2 = plt.subplot(312)
        sns.pointplot(x = 'Split (train:test)', y = 'Train_anomaly_percentage', 
                      hue = 'Dataset', data = dataset_info, ax=ax2)
        plt.title('Percentage of anomaly edges in training dataset')
        plt.ylabel('Anomaly share')
        ax2.get_legend().remove();
    
    #The anomaly % plot for test data:
    if show_test:
        ax3 = plt.subplot(313)
        sns.pointplot(x = 'Split (train:test)', y = 'Test_anomaly_percentage', 
                      hue = 'Dataset', data = dataset_info, ax=ax3)
        plt.title('Percentage of anomaly edges in test dataset')
        plt.ylabel('Anomaly share')
        ax3.get_legend().remove();
    
    #plt.savefig("Figures/auc_and_anomaly_percentage_" + df_name + ".png")
    plt.show();
    
    
#single_dataset_plot("DARPA", show_train=True, show_test=True)

In [ ]:
def plot_time_per_method(dataset: str):
    '''
    dataset: one of DATASETS
    '''
    
    df = pd.read_csv('./CSV/rav_change_s.csv')
    df['Size'] = df['Size'].astype(str)
    df['Time'] = df['Time'].apply(lambda x: log2(x))
    df['Method_full'] = df['Method'] + " + " + df['MIDAS']
    
    plt.figure(figsize=(8,2), dpi=100)
    sns.scatterplot(x = 'Size', y = 'Time', data = df[df['Dataset']==dataset], hue='Method_full')
    plt.legend(title='')
    plt.title('Running time of different sample sizes in dataset ' + dataset)
    plt.xlabel("Maximum size of the training graph (# unique edges)")
    plt.ylabel("log_2(Time) (in seconds)")
    
    if dataset + '.png' not in os.listdir('./Figures/Time_vs_sample_size/'):
        plt.savefig('./Figures/Time_vs_sample_size/' + dataset + '.png', bbox_inches='tight')
    
    plt.legend(bbox_to_anchor=(1,1))
    plt.show();
    
# for dataset in DATASETS:
#     plot_time_per_method(df, dataset)

plot_time_per_method(df, 'NYC_Taxi')

In [ ]:
def filter_test(X_test, y_test, G: nx.Graph) -> tuple:
    '''
    Filter out edges already present in G
    Filter out using a vertex never seen before in G
    '''
    
    good_rows = [(edge[0], edge[1], label) for (edge, label) in zip(X_test, y_test) if \
                 ((edge[0], edge[1]) not in G.edges) and (edge[0] in G and edge[1] in G)]
    
    print("Filtered out", len(y_test) - len(good_rows), "rows.")
    print("Filtered out", round((len(y_test) - len(good_rows))/len(y_test)*100, 3), "% of the original dataset.")
    
    return [(edge[0], edge[1]) for edge in good_rows], [edge[2] for edge in good_rows]